# Derivation using FIM

In [ ]:
# pytest-nb-as-test: default-all=False
# turn off testing for this notebook because some steps take a long time to run

## Symbolic Fisher Information Matrix Computation with Fast-Oscillation Approximations

This code symbolically computes the Fisher Information Matrix (FIM) for a damped sine model using `sympy`. It incorporates physical assumptions from signal processing to simplify the expressions, specifically removing contributions from high-frequency oscillations. The resulting FIM entries represent analytically integrated expressions for uncertainty propagation.

### Model Definition

The damped sine model is defined as:

```python
f = A * exp(-lambda_D * t) * sin(omega * t + phi)
```

where:

* `A`: amplitude
* `lambda_D`: damping rate
* `omega`: angular frequency
* `phi`: phase offset
* `t`: time

The parameters of interest are: `[A, lambda_D, omega, phi]`.

### Fisher Matrix Overview

The Fisher Information Matrix (FIM) is defined element-wise as:

$$
F_{ij} = \frac{N}{\sigma_m^2 T} \int_0^T \frac{\partial f}{\partial \theta_i} \frac{\partial f}{\partial \theta_j} dt
$$

This is implemented by computing symbolic derivatives, products, and integrals.

---

### Fast-Oscillation Approximations

Due to the highly oscillatory nature of the sine and cosine terms (with frequency `omega`), fast oscillating components like `sin(omega t + phi) * cos(omega t + phi)` or `sin^2(...)` can be approximated using their time averages over a period:

* $\sin^2(\omega t + \phi) \rightarrow 1/2$
* $\sin(\omega t + \phi) \cdot \cos(\omega t + \phi) \rightarrow 0$

These approximations are applied via the `apply_fast_oscillation_approximations` function which uses `sympy.sp.Wild` pattern matching.

The function also logs which subexpressions were replaced for debugging or validation.

---

### Simplified Integral Computation

The `compute_integral_pair_simplified` function handles:

* Symbolic differentiation of the model w\.r.t. each parameter
* Expansion and simplification of the product of derivatives
* Application of fast oscillation approximations
* Symbolic integration over time
* Scaling by \$\frac{R}{\sigma\_m^2}\$ where \$R = \frac{N}{T}\$

The result is a fully symbolic expression for each element of the Fisher matrix.

---

### Fisher Matrix Construction

Two versions of the matrix builder exist:

* `compute_fisher_matrix_serial_simplified` — single-threaded loop
* A future parallel version could use `multiprocessing` to compute matrix entries in parallel

---

### Interactive Inspection

The script includes a loop that prints the symbolic integrand for each FIM element, before and after applying the oscillation approximations. This is useful for debugging, verifying that terms are properly removed, and visually inspecting the complexity of symbolic expressions.

```python
for ii in range(4):
    for jj in range(4):
        print(f"FIM[{ii}, {jj}]:")
        df1 = diff(f, params[ii])
        df2 = diff(f, params[jj])
        product = expand(df1 * df2)
        print(product)
        product = apply_fast_oscillation_approximations(product, t, omega, phi)
        print(product)
```

---

### Usage Context

This code is useful in precision metrology, atomic physics, or control theory where you:

* Need analytic uncertainty propagation
* Work with oscillatory time-domain signals
* Can exploit known structure (e.g., periodicity) to reduce symbolic complexity

The technique balances computational tractability with analytic insight, making it well suited for systems with periodic or exponentially damped signals.


In [2]:
import sympy as sp
import sym_functions

# Symbols
t = sp.symbols("t", positive=True, real=True)
T = sp.symbols("T", positive=True, real=True, nonzero=True)
A = sp.symbols("A", positive=True, real=True, nonzero=True)
N = sp.symbols("N", positive=True, real=True, nonzero=True)
sigma_m = sp.symbols("sigma_m", positive=True, real=True, nonzero=True)
lambda_D = sp.symbols("lambda_D", real=True, positive=True, nonzero=True)
omega = sp.symbols("omega", real=True, positive=True, nonzero=True)
phi = sp.symbols("phi", real=True)

# Model
f = A * sp.exp(-lambda_D * t) * sp.sin(omega * t + phi)


# Parameter list
params = [A, lambda_D, omega, phi]
param_indices = {param: i for i, param in enumerate(params)}

In [3]:
from sympy import Matrix, symbols, integrate, diff, simplify, eye
from typing import Dict, Tuple
from multiprocessing import Pool, cpu_count

from typing import Tuple, List


def apply_fast_oscillation_approximations(
    expr: sp.Expr,
    t: sp.Symbol,
    omega: sp.Symbol,
    phi: sp.Symbol,
    print_replacements: bool = True,
) -> Tuple[sp.Expr, List[sp.Expr]]:
    """
    Apply fast-oscillation approximations like:
    sin^2(ωt ± φ) → 1/2, sin(ωt ± φ)*cos(ωt ± φ) → 0
    and record the replaced subexpressions.

    Returns:
        (simplified expression, list of matched expressions that were replaced)
    """

    replacements_log: List[sp.Expr] = []

    prefactor = sp.Wild("prefactor")  # matches anything
    sign_zero = sp.Wild("sign", properties=[lambda expr: expr in (0, 1, -1)])
    arg1 = omega * t + phi * sign_zero

    def log_and_replace(matched_expr: sp.Expr, replacement: sp.Expr):
        replacements_log.append(matched_expr)
        return replacement

    # sin^2(...)
    expr = expr.replace(
        prefactor * sp.sin(arg1) ** 2,
        lambda **kwargs: log_and_replace(sp.sin(arg1) ** 2, sp.Rational(1, 2)),
    )

    # cos^2(...)
    expr = expr.replace(
        prefactor * cos(arg1) ** 2,
        lambda **kwargs: log_and_replace(sp.sin(arg1) ** 2, sp.Rational(1, 2)),
    )

    # # General sin*cos pattern
    A = sp.Wild("A", properties=[lambda e: e.has(omega * t)])
    B = sp.Wild("B", exclude=[omega, t])
    pattern = prefactor * sp.sin(A + B) * sp.cos(A + B)

    expr = expr.replace(pattern, lambda **kwargs: log_and_replace(pattern, 0))

    if print_replacements and replacements_log:
        print("Fast oscillation approximations applied to:")
        for r in replacements_log:
            print("  ", r)

    return expr


# Update the integral computation function to include simplification inside the parallelized task
def compute_integral_pair_simplified(
    args: Tuple[sp.Expr, sp.Expr, sp.Symbol, sp.Symbol, sp.Symbol, sp.Symbol],
) -> sp.Expr:
    df1, df2, t, T, R, sigma_m, phi = args
    product = sp.expand(df1 * df2)
    product = sp.simplify(product)
    product = apply_fast_oscillation_approximations(product, t, omega, phi)
    integral = sp.integrate(product, (t, 0, T))
    scaled = integral * R / sigma_m**2
    expr = sp.expand(scaled)  # Expand basic algebra first
    expr = sp.expand_trig(expr)  # Expand trig products (e.g., sin^2)
    expr = sp.factor(expr)  # Pull out common exponentials
    expr = sp.cancel(expr)  # Clean up rational forms
    expr = sp.collect(expr, T)  # Group by powers of T (or lambda_D, etc.)
    # expr = sp.simplify(expr)
    return expr


def compute_fisher_matrix_serial_simplified(
    model: sp.Expr,
    parameters: list[sp.Symbol],
    t: sp.Symbol,
    T: sp.Symbol,
    sigma_m: sp.Symbol,
    N: sp.Symbol,
) -> sp.Matrix:
    """
    Computes the Fisher Information Matrix (FIM) without parallelism,
    using simplified symbolic integrals over all parameter pairs.

    Args:
        model: symbolic expression of the model function
        parameters: list of parameters to differentiate with respect to
        t: time symbol
        T: total acquisition time symbol
        sigma_m: measurement noise
        N: number of samples

    Returns:
        sympy.Matrix: symbolic Fisher Information Matrix
    """
    R = N / T
    n = len(parameters)

    entries: list[sp.Expr] = []

    for p1 in parameters:
        df1 = sp.diff(model, p1)
        for p2 in parameters:
            df2 = sp.diff(model, p2)
            result = compute_integral_pair_simplified((df1, df2, t, T, R, sigma_m, phi))
            entries.append(result)

    return sp.Matrix(n, n, entries)

# integrands

In [18]:
params

[A, lambda_D, omega, phi]

In [19]:
p1 = params[0]
p2 = params[0]
df1 = sp.diff(f, p1)
df2 = sp.diff(f, p2)
product = sp.expand(df1 * df2)
product

exp(-2*lambda_D*t)*sin(omega*t + phi)**2

In [20]:
df1 = sp.diff(f, params[0])
df2 = sp.diff(f, params[1])
product = sp.expand(df1 * df2)
product = sp.simplify(product)
product = apply_fast_oscillation_approximations(product, t, omega, phi)
integral = sp.integrate(product, (t, 0, T))
integral = sp.expand(integral)  # Expand basic algebra first
integral = sp.expand_trig(integral)  # Expand trig products (e.g., sin^2)
integral = sp.factor(integral)  # Pull out common exponentials
integral = sp.cancel(integral)  # Clean up rational forms
print(sp.latex(integral))

Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
\frac{\left(2 A T \lambda_{D} - A e^{2 T \lambda_{D}} + A\right) e^{- 2 T \lambda_{D}}}{8 \lambda_{D}^{2}}


In [21]:
# inspect the integerands

for ii in range(4):
    for jj in range(4):
        print(f"FIM[{ii}, {jj}]  [{str(params[ii])}, {str(params[jj])}]  :")
        df1 = sp.diff(f, params[ii])
        df2 = sp.diff(f, params[jj])
        print(f"df1:   {sp.latex(df1)}")
        print(f"df2:    {sp.latex(df2)}")
        product = sp.expand(df1 * df2)
        print(f"product:    {sp.latex(product)}")
        product = apply_fast_oscillation_approximations(product, t, omega, phi)
        print(product)
        print("\n")

FIM[0, 0]  [A, A]  :
df1:   e^{- \lambda_{D} t} \sin{\left(\omega t + \phi \right)}
df2:    e^{- \lambda_{D} t} \sin{\left(\omega t + \phi \right)}
product:    e^{- 2 \lambda_{D} t} \sin^{2}{\left(\omega t + \phi \right)}
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
exp(-2*lambda_D*t)/2


FIM[0, 1]  [A, lambda_D]  :
df1:   e^{- \lambda_{D} t} \sin{\left(\omega t + \phi \right)}
df2:    - A t e^{- \lambda_{D} t} \sin{\left(\omega t + \phi \right)}
product:    - A t e^{- 2 \lambda_{D} t} \sin^{2}{\left(\omega t + \phi \right)}
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
-A*t*exp(-2*lambda_D*t)/2


FIM[0, 2]  [A, omega]  :
df1:   e^{- \lambda_{D} t} \sin{\left(\omega t + \phi \right)}
df2:    A t e^{- \lambda_{D} t} \cos{\left(\omega t + \phi \right)}
product:    A t e^{- 2 \lambda_{D} t} \sin{\left(\omega t + \phi \right)} \cos{\left(\omega t + \phi \right)}
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)

In [22]:
ii = 0
jj = 2
print(f"FIM[{ii}, {jj}]:")
df1 = sp.diff(f, params[ii])
df2 = sp.diff(f, params[jj])
product = sp.expand(df1 * df2)
print(product)
product = apply_fast_oscillation_approximations(product, t, omega, phi)
print(product)
print("\n")

FIM[0, 2]:
A*t*exp(-2*lambda_D*t)*sin(omega*t + phi)*cos(omega*t + phi)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
0




# compute the FIM

In [23]:
# runtime ~2 s
FIM = compute_fisher_matrix_serial_simplified(f, params, t, T, sigma_m, N)

Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   prefactor_*sin(A_ + B_)*cos(A_ + B_)
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
   sin(omega*t + phi*sign_)**2
Fast oscillation approximations applied to:
  

In [24]:
FIM

Matrix([
[                          (N*exp(2*T*lambda_D) - N)*exp(-2*T*lambda_D)/(4*T*lambda_D*sigma_m**2),                                       (2*A*N*T*lambda_D - A*N*exp(2*T*lambda_D) + A*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**2*sigma_m**2),                                                                                                                                      0,                                                                                                          0],
[(2*A*N*T*lambda_D - A*N*exp(2*T*lambda_D) + A*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**2*sigma_m**2), (-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**3*sigma_m**2),                                                                                                                                      0,                                                                                                          0],
[                              

In [25]:
params

[A, lambda_D, omega, phi]

In [26]:
rename_map = {
    A: sp.Symbol("A"),
    N: sp.Symbol("N"),
    T: sp.Symbol("T"),
    sigma_m: sp.Symbol(r"\sigma"),
    lambda_D: sp.Symbol(r"\lambda"),
    omega: sp.Symbol(r"\omega"),
    phi: sp.Symbol(r"\phi"),
    t: sp.Symbol("t"),
}

FIM_subbed = FIM.subs(rename_map)
renamed_params: list[sp.Symbol] = [rename_map[p] for p in params]


def is_latex_symbol(s: str) -> bool:
    return s.startswith("\\")


def latex_nonzero_fim_named(
    F: sp.Matrix,
    params: list[sp.Symbol],
) -> str:
    """Generate LaTeX align* block for nonzero FIM entries using renamed symbols,
    only outputting upper triangle and referencing symmetric entries."""

    lines = []

    for i in range(F.rows):
        for j in range(i, F.cols):
            val = F[i, j]
            if val.equals(0):
                continue

            name_i = params[i]
            name_j = params[j]

            def wrap(sym: str) -> str:
                res = sym if is_latex_symbol(str(sym)) else rf"\text{{{sym}}}"
                return res

            subscript = f"{wrap(name_i)},\\,{wrap(name_j)}"
            lhs = f"F_{{{subscript}}}"

            if i == j:
                rhs = sp.latex(val)
            else:
                # Reference symmetric element, render as plain text
                sym_i = rf"\text{{F}}_{{\text{{{name_j}}},\,\text{{{name_i}}}}}"
                rhs = sp.latex(val) if i < j else sym_i

            lines.append(f"{lhs} &= {rhs} \\\\")

    return r"\begin{align*}" + "\n" + "\n".join(lines) + "\n" + r"\end{align*}"


latex_block = latex_nonzero_fim_named(FIM_subbed, renamed_params)
print(latex_block)

\begin{align*}
F_{\text{A},\,\text{A}} &= \frac{\left(N e^{2 T \lambda} - N\right) e^{- 2 T \lambda}}{4 T \lambda \sigma^{2}} \\
F_{\text{A},\,\lambda} &= \frac{\left(2 A N T \lambda - A N e^{2 T \lambda} + A N\right) e^{- 2 T \lambda}}{8 T \lambda^{2} \sigma^{2}} \\
F_{\lambda,\,\lambda} &= \frac{\left(- 2 A^{2} N T^{2} \lambda^{2} - 2 A^{2} N T \lambda + A^{2} N e^{2 T \lambda} - A^{2} N\right) e^{- 2 T \lambda}}{8 T \lambda^{3} \sigma^{2}} \\
F_{\omega,\,\omega} &= \frac{\left(- 2 A^{2} N T^{2} \lambda^{2} - 2 A^{2} N T \lambda + A^{2} N e^{2 T \lambda} - A^{2} N\right) e^{- 2 T \lambda}}{8 T \lambda^{3} \sigma^{2}} \\
F_{\omega,\,\phi} &= \frac{\left(- 2 A^{2} N T \lambda + A^{2} N e^{2 T \lambda} - A^{2} N\right) e^{- 2 T \lambda}}{8 T \lambda^{2} \sigma^{2}} \\
F_{\phi,\,\phi} &= \frac{\left(A^{2} N e^{2 T \lambda} - A^{2} N\right) e^{- 2 T \lambda}}{4 T \lambda \sigma^{2}} \\
\end{align*}


# inverse

In [27]:
import importlib
import sym_functions

importlib.reload(sym_functions)

<module 'sym_functions' from '/workspaces/repo/src/sym_functions.py'>

In [28]:
# runtime ~ 30s
FIM_inv = sym_functions.inverse_via_symmetric_substitution(FIM)

creating template for symbolic symmetric inverse
applying substitution to symbolic inverse


In [29]:
FIM_inv

Matrix([
[       (-(A**2*N*exp(2*T*lambda_D) - A**2*N)*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*exp(-6*T*lambda_D)/(256*T**3*lambda_D**7*sigma_m**6) + (-2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-6*T*lambda_D)/(512*T**3*lambda_D**7*sigma_m**6))/(-(N*exp(2*T*lambda_D) - N)*(A**2*N*exp(2*T*lambda_D) - A**2*N)*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*exp(-8*T*lambda_D)/(1024*T**4*lambda_D**8*sigma_m**8) + (N*exp(2*T*lambda_D) - N)*(-2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)**2*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-8*T*lambda_D)/(2048*T**4*lambda_D**8*sigma_m**8) + (A**2*N*exp(2*T*lambda_D) - A**2*N)*(2*A*N*T*lambda_D - A*N*exp(2*T*lambda_D) + A*N)**2*(-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*l

In [30]:
index_a = param_indices[A]
var_A = FIM_inv[index_a, index_a]
index_phi = param_indices[phi]
var_phi = FIM_inv[index_phi, index_phi]
index_omega = param_indices[omega]
var_omega = FIM_inv[index_omega, index_omega]
index_lambda = param_indices[lambda_D]
var_lambda = FIM_inv[index_lambda, index_lambda]

In [31]:
sym_functions.sym_info(var_lambda)

symbolic expression has 295 terms and length 1_188
inbuilt count:
  sp.pow                  64
  sp.trig                  0
symbols count:            10
  N                       45
  T                       41
  lambda_D                41
  A                       37
  sigma_m                  6
  1/2048                   2
  -1/1024                  1
  1/256                    1
  -1/4096                  1
  -1/128                   1


In [32]:
FIM_inv_symp = FIM_inv
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.expand)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.expand_trig)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.factor)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.cancel)
FIM_inv_symp = FIM_inv_symp.applyfunc(sp.simplify)

In [33]:
import sympy as sp

# Param list and symbolic variables
param_syms = [A, phi, omega, lambda_D]
param_vars = {}

# Rewrite and simplify each diagonal entry
for param in param_syms:
    idx = param_indices[param]
    var = FIM_inv_symp[idx, idx]

    var_hyper = var.rewrite(sp.cosh)
    var_hyper = var_hyper.rewrite(sp.cos).rewrite(sp.cosh)
    var_simplified = sp.simplify(var_hyper)

    param_vars[param] = var_simplified

In [34]:
from IPython.display import display, Math

# Loop through and display each simplified diagonal variance
for param, expr in param_vars.items():
    display(Math(rf"\sigma^2 ({sp.latex(param)} ) = {sp.latex(expr)}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [60]:
param_vars[A]

8*T*lambda_D*sigma_m**2*(T**2*lambda_D**2 + T*lambda_D - exp(2*T*lambda_D)/2 + 1/2)/(N*(2*T**2*lambda_D**2 - cosh(2*T*lambda_D) + 1))

In [61]:
sym_functions.sym_info(var_lambda)

symbolic expression has 36 terms and length 163
inbuilt count:
  sp.pow                   7
  sp.trig                  0
symbols count:             5
  lambda_D                 7
  T                        7
  N                        1
  A                        1
  sigma_m                  1


In [68]:
symbols

<function sympy.core.symbol.symbols(names, *, cls=<class 'sympy.core.symbol.Symbol'>, **args) -> 'Any'>

In [72]:
# generate code

for symbol in params:
    print(f"var {str(symbol)}")
    # Map symbols to variable names in function signature
    rename_map = {
        A: "amp",
        N: "samp_num",
        T: "samp_time",
        sigma_m: "sigma_obs",
        lambda_D: "damp_rate",
        omega: "omega",  # leave unchanged
        phi: "phi",  # leave unchanged
        t: "t",  # used internally, not in function signature
    }

    # Convert to substituted code string
    expr = param_vars[symbol]
    rename_symb_map = {k: sp.Symbol(v) for k, v in rename_map.items()}
    expr_sub = expr.subs(rename_symb_map)
    pycode_expr = sp.printing.pycode(expr_sub)
    print(pycode_expr)
    print("\n")

var A
8*damp_rate*samp_time*sigma_obs**2*(damp_rate**2*samp_time**2 + damp_rate*samp_time - 1/2*math.exp(2*damp_rate*samp_time) + 1/2)/(samp_num*(2*damp_rate**2*samp_time**2 - math.cosh(2*damp_rate*samp_time) + 1))


var lambda_D
-8*damp_rate**3*samp_time*sigma_obs**2*(math.exp(2*damp_rate*samp_time) - 1)/(amp**2*samp_num*(2*damp_rate**2*samp_time**2 - math.cosh(2*damp_rate*samp_time) + 1))


var omega
-8*damp_rate**3*samp_time*sigma_obs**2*(math.exp(2*damp_rate*samp_time) - 1)/(amp**2*samp_num*(2*damp_rate**2*samp_time**2 - math.cosh(2*damp_rate*samp_time) + 1))


var phi
8*damp_rate*samp_time*sigma_obs**2*(damp_rate**2*samp_time**2 + damp_rate*samp_time - 1/2*math.exp(2*damp_rate*samp_time) + 1/2)/(amp**2*samp_num*(2*damp_rate**2*samp_time**2 - math.cosh(2*damp_rate*samp_time) + 1))




In [ ]:
# generate latex

for symbol in params:
    print(f"var {str(symbol)}")
    # Map symbols to variable names in function signature
    rename_map = {
        A: sp.Symbol("A"),
        N: sp.Symbol("N"),
        T: sp.Symbol("T"),
        sigma_m: sp.Symbol(r"\sigma"),
        lambda_D: sp.Symbol(r"\lambda"),
        omega: sp.Symbol(r"\omega"),
        phi: sp.Symbol(r"\phi"),
        t: sp.Symbol("t"),
    }

    # Convert to substituted code string
    expr = param_vars[symbol]
    rename_symb_map = {k: v for k, v in rename_map.items()}
    expr_sub = expr.subs(rename_symb_map)
    latex_expr = sp.latex(expr_sub)
    print(latex_expr)
    print("\n")

var A
\frac{8 T \lambda \sigma^{2} \left(T^{2} \lambda^{2} + T \lambda - \frac{e^{2 T \lambda}}{2} + \frac{1}{2}\right)}{N \left(2 T^{2} \lambda^{2} - \cosh{\left(2 T \lambda \right)} + 1\right)}


var lambda_D
- \frac{8 T \lambda^{3} \sigma^{2} \left(e^{2 T \lambda} - 1\right)}{A^{2} N \left(2 T^{2} \lambda^{2} - \cosh{\left(2 T \lambda \right)} + 1\right)}


var omega
- \frac{8 T \lambda^{3} \sigma^{2} \left(e^{2 T \lambda} - 1\right)}{A^{2} N \left(2 T^{2} \lambda^{2} - \cosh{\left(2 T \lambda \right)} + 1\right)}


var phi
\frac{8 T \lambda \sigma^{2} \left(T^{2} \lambda^{2} + T \lambda - \frac{e^{2 T \lambda}}{2} + \frac{1}{2}\right)}{A^{2} N \left(2 T^{2} \lambda^{2} - \cosh{\left(2 T \lambda \right)} + 1\right)}




In [ ]:
from sympy import latex

FIM_subbed = FIM.subs(rename_map)
renamed_params: list[sp.Symbol] = [rename_map[p] for p in params]


def is_latex_symbol(s: str) -> bool:
    return s.startswith("\\")


def latex_nonzero_fim_named(
    F: sp.Matrix,
    params: list[sp.Symbol],
) -> str:
    """Generate LaTeX align* block for nonzero FIM entries using renamed symbols,
    only outputting upper triangle and referencing symmetric entries."""

    lines = []

    for i in range(F.rows):
        for j in range(i, F.cols):
            val = F[i, j]
            if val.equals(0):
                continue

            name_i = params[i]
            name_j = params[j]

            def wrap(sym: str) -> str:
                res = sym if is_latex_symbol(str(sym)) else rf"\text{{{sym}}}"
                return res

            subscript = f"{wrap(name_i)},\\,{wrap(name_j)}"
            lhs = f"F_{{{subscript}}}"

            if i == j:
                rhs = latex(val)
            else:
                # Reference symmetric element, render as plain text
                sym_i = rf"\text{{F}}_{{\text{{{name_j}}},\,\text{{{name_i}}}}}"
                rhs = latex(val) if i < j else sym_i

            lines.append(f"{lhs} &= {rhs} \\\\")

    return r"\begin{align*}" + "\n" + "\n".join(lines) + "\n" + r"\end{align*}"


latex_block = latex_nonzero_fim_named(FIM_subbed, renamed_params)
print(latex_block)

# limiting case

In [92]:
import numpy as np

np.sqrt(8) / np.sqrt(2)

np.float64(2.0)

In [97]:
for symbol in params:
    print(f"var {str(symbol)}")
    # Map symbols to variable names in function signature
    rename_map = {
        A: sp.Symbol("A"),
        N: sp.Symbol("N"),
        T: sp.Symbol("T"),
        sigma_m: sp.Symbol(r"\sigma"),
        lambda_D: sp.Symbol(r"\lambda"),
        omega: sp.Symbol(r"\omega"),
        phi: sp.Symbol(r"\phi"),
        t: sp.Symbol("t"),
    }

    # Convert to substituted code string
    expr = param_vars[symbol]

    lim_expr = sp.limit(expr, lambda_D, 0)
    rename_symb_map = {k: v for k, v in rename_map.items()}
    expr_sub = lim_expr.subs(rename_symb_map)
    latex_expr = sp.latex(expr_sub)
    print(latex_expr)
    print("\n")

var A
\frac{8 \sigma^{2}}{N}


var lambda_D
\frac{24 \sigma^{2}}{A^{2} N T^{2}}


var omega
\frac{24 \sigma^{2}}{A^{2} N T^{2}}


var phi
\frac{8 \sigma^{2}}{A^{2} N}




In [94]:
fist_lim = sp.limit(param_vars[A], T, 0)
print(sp.latex(fist_lim))

\frac{8 \sigma_{m}^{2}}{N}


# optimum duration

In [ ]:
import copy

# try to non-diemsnoonalize the expression
simp_expr = copy.deepcopy(var_omega)
c1 = sp.symbols("c1", positive=True, real=True)
ratio_expr = simp_expr.subs(T, c1 / lambda_D)
c2 = sp.symbols("c2", positive=True, real=True)
ratio_expr = ratio_expr.subs(sigma_m, c2 * A)
ratio_expr = ratio_expr.simplify() * N / 16 / lambda_D**2

In [ ]:
d_expr = sp.diff(ratio_expr, c1)
d_expr = d_expr / c2**2
d_expr.simplify()

(-4*c1*(1 - exp(2*c1))*(2*c1**2 + 2*c1 - exp(2*c1) + 1)*exp(2*c1) + (2*c1*(1 - exp(2*c1)) - 2*c1*exp(2*c1) - exp(2*c1) + 1)*(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1))*exp(2*c1)/(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1)**2

In [ ]:
import scipy
from scipy.optimize import minimize_scalar


sp.solve(d_expr_sub, c1)

NotImplementedError: multiple generators [c1, exp(c1)]
No algorithms are implemented to solve equation c1*(1 - exp(2*c1))*(-8*c1**2*exp(2*c1) - 8*c1*exp(2*c1) + 4*exp(4*c1) - 4*exp(2*c1))*exp(2*c1)/(100*(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1)**2) + c1*(1 - exp(2*c1))*exp(2*c1)/(50*(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1)) - c1*exp(4*c1)/(50*(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1)) + (1 - exp(2*c1))*exp(2*c1)/(100*(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1))

In [ ]:
from mpmath import findroot
from mpmath import mp

mp.dps = 50  # Use 50 decimal digits of precision globally

# Lambdify the derivative for numerical solving
f_mpmath = sp.lambdify(c1, d_expr_sub, modules="mpmath")


root = findroot(f_mpmath, 2.0, solver="newton")
print(f"Root at c1 ≈ {root}")

Root at c1 ≈ 2.0174775197974972412351272762766629167709465824412


In [ ]:
epsilons = [1e-10, 1e-20, 1e-30, 1e-40]
roots = [findroot(f_mpmath, 2.0, tol=eps, solver="newton") for eps in epsilons]
for eps, r in zip(epsilons, roots):
    print(f"tol={eps}: c1 ≈ {r}")

tol=1e-10: c1 ≈ 2.0174775197974972412351272762759718726768980233288
tol=1e-20: c1 ≈ 2.0174775197974972412351272762766629167709465824412
tol=1e-30: c1 ≈ 2.0174775197974972412351272762766629167709465824412
tol=1e-40: c1 ≈ 2.0174775197974972412351272762766629167709465824412


## Damping ratio

In [ ]:
import copy

# try to non-diemsnoonalize the expression
simp_expr = copy.deepcopy(var_lambda)
c1 = sp.symbols("c1", positive=True, real=True)
ratio_expr = simp_expr.subs(T, c1 / lambda_D)
c2 = sp.symbols("c2", positive=True, real=True)
ratio_expr = ratio_expr.subs(sigma_m, c2 * A)
ratio_expr = ratio_expr.simplify() * N / 16 / lambda_D**2
ratio_expr

c1*c2**2*(1 - exp(2*c1))*exp(2*c1)/(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1)

In [ ]:
d_expr = sp.diff(ratio_expr, c1)
d_expr = d_expr / c2**2
d_expr.simplify()

(-4*c1*(1 - exp(2*c1))*(2*c1**2 + 2*c1 - exp(2*c1) + 1)*exp(2*c1) + (2*c1*(1 - exp(2*c1)) - 2*c1*exp(2*c1) - exp(2*c1) + 1)*(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1))*exp(2*c1)/(4*c1**2*exp(2*c1) - exp(4*c1) + 2*exp(2*c1) - 1)**2

In [ ]:
from mpmath import findroot
from mpmath import mp

mp.dps = 50  # Use 50 decimal digits of precision globally

# Lambdify the derivative for numerical solving
f_mpmath = sp.lambdify(c1, d_expr_sub, modules="mpmath")


root = findroot(f_mpmath, 2.0, solver="newton")
print(f"Root at c1 ≈ {root}")

Root at c1 ≈ 2.0174775197974972412351272762766629167709465824412


In [ ]:
root

mpf('2.0174775197974972412351272762766629167709465824412037')

In [ ]:
var_omega

16*T*lambda_D**3*sigma_m**2*(1 - exp(2*T*lambda_D))*exp(2*T*lambda_D)/(A**2*N*(4*T**2*lambda_D**2*exp(2*T*lambda_D) - exp(4*T*lambda_D) + 2*exp(2*T*lambda_D) - 1))

In [ ]:
best_var_omega = var_omega.subs(T, 2.0 / lambda_D)

In [ ]:
sp.sqrt(best_var_omega) / (2 * np.pi)

1.08926063665112*lambda_D*sigma_m/(A*sqrt(N))

In [ ]:
best_var_lambda = var_lambda.subs(T, 2.0 / lambda_D)
sp.sqrt(best_var_lambda)

6.84402642789539*lambda_D*sigma_m/(A*sqrt(N))

# truncated inverse
If you want to see what the unc does when you ignore terms

In [ ]:
from typing import List, Tuple


def zero_except(mat_in: sp.Matrix, keep_pairs: List[Tuple[int, int]]) -> sp.Matrix:
    """
    Returns a copy of the matrix with all off-diagonal elements zeroed out,
    except those in the keep_pairs list (applied symmetrically).

    Args:
        F: SymPy square matrix
        keep_pairs: list of (i, j) index pairs to preserve (both [i,j] and [j,i])

    Returns:
        A new matrix with only specified off-diagonals (and the diagonal) preserved.
    """
    n = mat_in.shape[0]
    assert mat_in.shape == (n, n), "Matrix must be square"

    # Build symmetric set of allowed pairs
    keep = {(i, j) for (i, j) in keep_pairs}
    keep |= {(j, i) for (i, j) in keep_pairs}

    F_new = mat_in.copy()
    for i in range(n):
        for j in range(n):
            if i == j:
                continue  # Always keep diagonal
            if (i, j) not in keep:
                F_new[i, j] = 0
    return F_new


FIM_simplified = FIM.copy()
allowed_pairs = []
index_pairs = [(param_indices[a], param_indices[b]) for a, b in allowed_pairs]
FIM_simplified = zero_except(FIM_simplified, index_pairs)
FIM_simplified

Matrix([
[(N*exp(2*T*lambda_D) - N)*exp(-2*T*lambda_D)/(4*T*lambda_D*sigma_m**2),                                                                                                                                      0,                                                                                                                                      0,                                                                                0],
[                                                                     0, (-2*A**2*N*T**2*lambda_D**2 - 2*A**2*N*T*lambda_D + A**2*N*exp(2*T*lambda_D) - A**2*N)*exp(-2*T*lambda_D)/(8*T*lambda_D**3*sigma_m**2),                                                                                                                                      0,                                                                                0],
[                                                                     0,                                                               

In [ ]:
# runtime ~ 30s
FIM_simplified_inv = sym_functions.inverse_via_symmetric_substitution(FIM_simplified)

creating template for symbolic symmetric inverse
applying substitution to symbolic inverse


In [ ]:
var_A = FIM_simplified_inv[index_a, index_a]
sym_functions.sym_info(var_A)

symbolic expression has 14 terms and length 67
inbuilt count:
  sp.pow                   2
  sp.trig                  0
symbols count:             4
  lambda_D                 3
  T                        3
  N                        2
  sigma_m                  1


In [ ]:
std_A = sp.sqrt(var_A).simplify()
std_A

2*sqrt(T)*sqrt(lambda_D)*sigma_m*sqrt(1/(exp(2*T*lambda_D) - 1))*exp(T*lambda_D)/sqrt(N)

# when $T$ is less than the period 

In [ ]:
# number of cycles sampled
R = sp.symbols("R", positive=True, real=True, nonzero=True)

In [8]:
def apply_approx_for_short_time(
    expr: sp.Expr,
    t: sp.Symbol,
    omega: sp.Symbol,
    phi: sp.Symbol,
    print_replacements: bool = True,
) -> Tuple[sp.Expr, List[sp.Expr]]:
    """
    Apply fast-oscillation approximations like:
    sin^2(ωt ± φ) → 1/2, sin(ωt ± φ)*cos(ωt ± φ) → 0
    and record the replaced subexpressions.

    Returns:
        (simplified expression, list of matched expressions that were replaced)
    """

    replacements_log: List[sp.Expr] = []

    prefactor = sp.Wild("prefactor")  # matches anything

    def log_and_replace(matched_expr: sp.Expr, replacement: sp.Expr):
        replacements_log.append(matched_expr)
        return replacement

    # Match exp(-k * lambda_D * t) for any positive k
    k = sp.Wild(
        "k", properties=[lambda expr: expr.is_Number or expr.is_integer or expr.is_real]
    )
    exp_pattern_general = sp.exp(-k * lambda_D * t)

    expr = expr.replace(
        prefactor * exp_pattern_general,
        lambda prefactor, k: log_and_replace(exp_pattern_general.subs(k, k), prefactor),
    )

    if print_replacements and replacements_log:
        print("approximations applied to:")
        for r in set(replacements_log):
            print("  ", r)

    return expr


def simplify_trig_identities(
    expr: sp.Expr, phi: sp.Symbol, print_replacements: bool = True
) -> Tuple[sp.Expr, List[sp.Expr]]:
    """
    Simplifies trig identities with explicit logging via log_and_replace().
    """
    replacements_log: List[sp.Expr] = []

    a = sp.Wild("a", exclude=[phi, sp.sin(phi), sp.cos(phi)])

    def log_and_replace(matched_expr: sp.Expr, replacement: sp.Expr):
        replacements_log.append((matched_expr, replacement))
        return replacement

    # Pattern: a * sin^2(phi) * cos^2(phi) => a * (1 - cos(4phi))/8
    pattern = a * sp.sin(phi) ** 2 * sp.cos(phi) ** 2
    expr = expr.replace(
        pattern, lambda a: log_and_replace(pattern, a * (1 - sp.cos(4 * phi)) / 8)
    )

    # Pattern: a * sin(phi) * cos(phi) => a * sin(2phi)/2
    pattern = a * sp.sin(phi) * sp.cos(phi)
    expr = expr.replace(
        pattern, lambda a: log_and_replace(pattern, a * sp.sin(2 * phi) / 2)
    )

    if print_replacements and replacements_log:
        print("Trig identity simplifications applied to:")
        for old, new in replacements_log:
            print("  ", old, "→", new)

    return expr.simplify()

In [14]:
params

[A, lambda_D, omega, phi]

In [ ]:
print("test")
ii = 2
for jj in range(4):
    print(f"FIM[{ii}, {jj}]  [{str(params[ii])}, {str(params[jj])}]  :")
    df1 = sp.diff(f, params[ii])
    df2 = sp.diff(f, params[jj])
    print(f"df1:   {df1}")
    print(f"df2:    {df2}")
    product = sp.expand(df1 * df2)
    print(f"product:    {product}")
    product = apply_approx_for_short_time(product, t, omega, phi)
    print(f"product:    {product}")
    product = simplify_trig_identities(product, phi)
    product = sp.expand(product)
    # product=product.series(t, 0, 2).removeO().expand()
    print(product)
    print("\n")

In [25]:
# Update the integral computation function to include simplification inside the parallelized task
def compute_integral_pair_simplified(
    args: Tuple[sp.Expr, sp.Expr, sp.Symbol, sp.Symbol, sp.Symbol, sp.Symbol],
) -> sp.Expr:

    expansion_pt = sp.Rational(1, 2) * 1 / (2 * sp.pi * omega)

    df1, df2, t, T, R, sigma_m, phi = args
    product = sp.expand(df1 * df2)
    product = sp.simplify(product)
    product = apply_approx_for_short_time(product, t, omega, phi)
    product = simplify_trig_identities(product, phi)
    integral = sp.integrate(product, (t, 0, T))
    integral = sp.expand(integral)
    integral = simplify_trig_identities(integral, phi)
    integral.series(T, expansion_pt, 5).removeO().expand()
    scaled = integral * R / sigma_m**2
    expr = sp.expand(scaled)  # Expand basic algebra first
    expr = sp.expand_trig(expr)  # Expand trig products (e.g., sin^2)
    expr = sp.factor(expr)  # Pull out common exponentials
    expr = sp.cancel(expr)  # Clean up rational forms
    expr = sp.collect(expr, T)  # Group by powers of T (or lambda_D, etc.)
    # expr = sp.simplify(expr)
    return expr


def compute_fisher_matrix_serial_simplified(
    model: sp.Expr,
    parameters: list[sp.Symbol],
    t: sp.Symbol,
    T: sp.Symbol,
    sigma_m: sp.Symbol,
    N: sp.Symbol,
) -> sp.Matrix:
    """
    Computes the Fisher Information Matrix (FIM) without parallelism,
    using simplified symbolic integrals over all parameter pairs.

    Args:
        model: symbolic expression of the model function
        parameters: list of parameters to differentiate with respect to
        t: time symbol
        T: total acquisition time symbol
        sigma_m: measurement noise
        N: number of samples

    Returns:
        sympy.Matrix: symbolic Fisher Information Matrix
    """
    R = N / T
    n = len(parameters)

    entries: list[sp.Expr] = []

    for p1 in parameters:
        df1 = sp.diff(model, p1)
        for p2 in parameters:
            df2 = sp.diff(model, p2)
            result = compute_integral_pair_simplified((df1, df2, t, T, R, sigma_m, phi))
            entries.append(result)
            print(f"F[{p1},{p2}] = {result}\n")

    return sp.Matrix(n, n, entries)


# runtime ~2 s
FIM_short_time = compute_fisher_matrix_serial_simplified(f, params, t, T, sigma_m, N)

approximations applied to:
   exp(-lambda_D*t*k_)
Trig identity simplifications applied to:
   a_*sin(phi)*cos(phi) → sin(2*phi)/(4*omega)
F[A,A] = (N*T*omega - 2*N*sin(phi)*cos(phi)*cos(T*omega)**2 + 2*N*sin(phi)*cos(phi) - 2*N*sin(T*omega)*cos(phi)**2*cos(T*omega) + N*sin(T*omega)*cos(T*omega))/(2*T*omega*sigma_m**2)

approximations applied to:
   exp(-lambda_D*t*k_)
F[A,lambda_D] = (-A*N*T**2*omega**2 + A*N*sin(phi)**2*sin(T*omega)**2 - 2*A*N*sin(phi)*sin(T*omega)*cos(phi)*cos(T*omega) + A*N*cos(phi)**2*cos(T*omega)**2 - A*N*cos(phi)**2 + T*(4*A*N*omega*sin(phi)*cos(phi)*cos(T*omega)**2 - 2*A*N*omega*sin(phi)*cos(phi) + 4*A*N*omega*sin(T*omega)*cos(phi)**2*cos(T*omega) - 2*A*N*omega*sin(T*omega)*cos(T*omega)))/(4*T*omega**2*sigma_m**2)

approximations applied to:
   exp(-lambda_D*t*k_)
F[A,omega] = (2*A*N*sin(phi)*cos(phi)*cos(T*omega)**2 - 2*A*N*sin(phi)*cos(phi) + 2*A*N*sin(T*omega)*cos(phi)**2*cos(T*omega) - A*N*sin(T*omega)*cos(T*omega) + T*(4*A*N*omega*sin(phi)*sin(T*omega)*cos

In [17]:
FIM_short_time

Matrix([
[                                                                 (-2*N*sin(phi)*cos(1/pi)*cos(phi) + 2*pi*N*sin(1/pi)*sin(phi)*cos(phi) - 2*pi*N*cos(1/pi)*cos(phi)**2 - 2*N*sin(1/pi)*cos(phi)**2 + N*sin(1/pi) + pi*N*cos(1/pi) + pi*N + T*(2*pi*N*omega*sin(phi)*cos(1/pi)*cos(phi) + 2*pi*N*omega*sin(1/pi)*cos(phi)**2 - pi*N*omega*sin(1/pi)))/(2*pi*sigma_m**2),                                                                                                                                                      (2*A*N*sin(phi)*cos(1/pi)*cos(phi) + 2*A*N*sin(1/pi)*cos(phi)**2 - A*N*sin(1/pi) + T*(-2*pi**2*A*N*omega*sin(1/pi)*sin(phi)*cos(phi) - 2*pi*A*N*omega*sin(phi)*cos(1/pi)*cos(phi) - 2*pi*A*N*omega*sin(1/pi)*cos(phi)**2 + 2*pi**2*A*N*omega*cos(1/pi)*cos(phi)**2 - pi**2*A*N*omega - pi**2*A*N*omega*cos(1/pi) + pi*A*N*omega*sin(1/pi)))/(4*pi**2*omega*sigma_m**2),                                                                                                                          

In [26]:
# runtime ~ 30s
import sym_functions

FIM_short_time_inv = sym_functions.inverse_via_symmetric_substitution(FIM_short_time)

creating template for symbolic symmetric inverse
applying substitution to symbolic inverse


In [169]:
FIM_short_time_inv

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [23]:
FIM_short_time_inv[0, 0]

(-(A**2*N*T*omega + 2*A**2*N*sin(phi)*cos(phi)*cos(T*omega)**2 - 2*A**2*N*sin(phi)*cos(phi) + 2*A**2*N*sin(T*omega)*cos(phi)**2*cos(T*omega) - A**2*N*sin(T*omega)*cos(T*omega))*(2*A**2*N*T**3*omega**3 - 6*A**2*N*sin(phi)*cos(phi)*cos(T*omega)**2 + 6*A**2*N*sin(phi)*cos(phi) - 6*A**2*N*sin(T*omega)*cos(phi)**2*cos(T*omega) + 3*A**2*N*sin(T*omega)*cos(T*omega) + T**2*(12*A**2*N*omega**2*sin(phi)*cos(phi)*cos(T*omega)**2 - 6*A**2*N*omega**2*sin(phi)*cos(phi) + 12*A**2*N*omega**2*sin(T*omega)*cos(phi)**2*cos(T*omega) - 6*A**2*N*omega**2*sin(T*omega)*cos(T*omega)) + T*(-12*A**2*N*omega*sin(phi)*sin(T*omega)*cos(phi)*cos(T*omega) + 12*A**2*N*omega*cos(phi)**2*cos(T*omega)**2 - 6*A**2*N*omega*cos(phi)**2 - 6*A**2*N*omega*cos(T*omega)**2 + 3*A**2*N*omega))*(2*A**2*N*T**3*omega**3 + 6*A**2*N*sin(phi)*cos(phi)*cos(T*omega)**2 - 6*A**2*N*sin(phi)*cos(phi) + 6*A**2*N*sin(T*omega)*cos(phi)**2*cos(T*omega) - 3*A**2*N*sin(T*omega)*cos(T*omega) + T**2*(-12*A**2*N*omega**2*sin(phi)*cos(phi)*cos(T*omega

In [27]:
# Param list and symbolic variables
param_syms = [A, phi, omega, lambda_D]
param_vars = {}

R = sp.symbols("R", positive=True)

# Rewrite and simplify each diagonal entry
for param in param_syms:
    idx = param_indices[param]
    expr = FIM_short_time_inv[idx, idx]
    expr_sub = expr.subs(T * omega, R)
    expr = sp.collect(expr, R)
    var_simplified = sp.simplify(expr)

    param_vars[param] = var_simplified
    print(f"done {param}")

KeyboardInterrupt: 

In [ ]:
from IPython.display import display, Math

# Loop through and display each simplified diagonal variance
for param, expr in param_vars.items():
    display(Math(rf"\sigma^2 ({sp.latex(param)} ) = {sp.latex(expr)}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
# generate code

for symbol in params:
    print(f"var {str(symbol)}")
    # Map symbols to variable names in function signature
    rename_map = {
        A: "amp",
        N: "samp_num",
        T: "samp_time",
        sigma_m: "sigma_obs",
        lambda_D: "damp_rate",
        omega: "omega",  # leave unchanged
        phi: "phi",  # leave unchanged
        t: "t",  # used internally, not in function signature
    }

    # Convert to substituted code string
    expr = param_vars[symbol]
    rename_symb_map = {k: sp.Symbol(v) for k, v in rename_map.items()}
    expr_sub = expr.subs(rename_symb_map)
    pycode_expr = sp.printing.pycode(expr_sub)
    print(pycode_expr)
    print("\n")

var A
-1/2*omega*samp_time*sigma_obs**2*(18*(2*omega*samp_time - math.sin(2*phi) + math.sin(2*omega*samp_time + 2*phi))*(2*omega**2*samp_time**2*math.cos(2*omega*samp_time + 2*phi) - 2*omega*samp_time*math.sin(2*omega*samp_time + 2*phi) + math.cos(2*phi) - math.cos(2*omega*samp_time + 2*phi))**2 + 2*(2*omega*samp_time - math.sin(2*phi) + math.sin(2*omega*samp_time + 2*phi))*(-4*omega**3*samp_time**3 + 6*omega**2*samp_time**2*math.sin(2*omega*samp_time + 2*phi) + 6*omega*samp_time*math.cos(2*omega*samp_time + 2*phi) + 3*math.sin(2*phi) - 3*math.sin(2*omega*samp_time + 2*phi))*(4*omega**3*samp_time**3 + 6*omega**2*samp_time**2*math.sin(2*omega*samp_time + 2*phi) + 6*omega*samp_time*math.cos(2*omega*samp_time + 2*phi) + 3*math.sin(2*phi) - 3*math.sin(2*omega*samp_time + 2*phi)) + 3*(2*omega*samp_time*math.cos(2*omega*samp_time + 2*phi) + math.sin(2*phi) - math.sin(2*omega*samp_time + 2*phi))**2*(4*omega**3*samp_time**3 + 6*omega**2*samp_time**2*math.sin(2*omega*samp_time + 2*phi) + 6*omeg

# OLD

In [ ]:
def compare_with_target(expressions: Dict[str, sp.Expr]) -> Dict[str, bool]:
    """Compare derived variances with the target formulas from the derivation.

    Parameters
    ----------
    expressions : Dict[str, sp.Expr]
        Dictionary containing the derived variances ``var_A``, ``var_phi``
        and ``var_omega``.

    Returns
    -------
    Dict[str, bool]
        Dictionary indicating whether each derived expression is
        algebraically equivalent to the corresponding target expression.
    """

    lambda_D, T, A, sigma_m, N = sp.symbols("lambda_D T A sigma_m N", positive=True)
    # Target expressions from the supplied derivation (see Eqs. (A.36)–(A.38)).
    var_A_target: sp.Expr = sp.simplify(
        (2 * lambda_D * sigma_m**2 * T * (sp.coth(lambda_D * T) + 1)) / N
    )
    var_phi_target: sp.Expr = sp.simplify(
        (
            4
            * lambda_D
            * sigma_m**2
            * T
            * (-2 * lambda_D * T * (lambda_D * T + 1) + sp.exp(2 * lambda_D * T) - 1)
        )
        / (A**2 * N * (-2 * lambda_D**2 * T**2 + sp.cosh(2 * lambda_D * T) - 1))
    )
    var_omega_target: sp.Expr = sp.simplify(
        (8 * lambda_D**3 * sigma_m**2 * T * (sp.exp(2 * lambda_D * T) - 1))
        / (A**2 * N * (-2 * lambda_D**2 * T**2 + sp.cosh(2 * lambda_D * T) - 1))
    )

    # Substitution dictionary for matching symbols in derived expressions
    subs_dict = {
        sp.symbols("lambda_D", positive=True): lambda_D,
        sp.symbols("T", positive=True): T,
        sp.symbols("A", positive=True): A,
        sp.symbols("sigma_m", positive=True): sigma_m,
        sp.symbols("N", positive=True): N,
    }

    # Compare via ratio: two expressions are equivalent if their ratio simplifies to 1
    def match(expr1: sp.Expr, expr2: sp.Expr) -> bool:
        """Return True if two expressions are equivalent.

        The comparison first attempts symbolic simplification of the ratio
        ``expr1/expr2`` to 1.  If that fails (e.g. due to latent
        exponential–hyperbolic identities), a numerical check is
        performed at several positive values of ``lambda_D`` and ``T``.
        """
        ratio = sp.simplify(expr1 / expr2)
        # Direct symbolic check
        if sp.simplify(ratio - 1) == 0:
            return True
        # Numerical sampling fallback
        numeric_tests = [(0.1, 2.0), (0.5, 1.2), (0.2, 3.5)]
        lambda_D_sym, T_sym = sp.symbols("lambda_D T", positive=True)
        f_lambda = lambda_D_sym
        f_T = T_sym
        # Create callable function for ratio using lambdify to ensure numeric evaluation
        ratio_func = sp.lambdify((lambda_D_sym, T_sym), ratio, "numpy")
        import math

        for lam_val, T_val in numeric_tests:
            try:
                val = ratio_func(lam_val, T_val)
            except Exception:
                return False
            if not math.isfinite(val) or abs(val - 1.0) > 1e-10:
                return False
        return True

    var_A_match: bool = match(expressions["var_A"].subs(subs_dict), var_A_target)
    var_phi_match: bool = match(expressions["var_phi"].subs(subs_dict), var_phi_target)
    var_omega_match: bool = match(
        expressions["var_omega"].subs(subs_dict), var_omega_target
    )

    return {
        "var_A": var_A_match,
        "var_phi": var_phi_match,
        "var_omega": var_omega_match,
    }